In [1]:
from transformers import BertTokenizer, BertForSequenceClassification

# Load the pre-trained BERT model and tokenizer
model = BertForSequenceClassification.from_pretrained('/home/j597s263/bert_sentiment_model')
tokenizer = BertTokenizer.from_pretrained('/home/j597s263/bert_sentiment_model')

# Set the model to evaluation mode
model.eval()

print("Model and tokenizer loaded successfully!")


OSError: Incorrect path_or_model_id: '/home/j597s263/bert_sentiment_model'. Please provide either the path to a local folder or the repo_id of a model on the Hub.

In [ ]:
###DOES NOT WORK###

'''
def classifier(texts):
    # Tokenize input texts
    inputs = tokenizer(texts, padding=True, truncation=True, max_length=512, return_tensors="pt")

    # Perform a forward pass through the model and get logits
    with torch.no_grad():
        outputs = model(**inputs)

    # Convert logits to probabilities
    probabilities = torch.nn.functional.softmax(outputs.logits, dim=-1).cpu().numpy()

    # Create a list of labels
    labels = ["NEGATIVE", "POSITIVE"]

    # Format the output in the same structure as the pipeline
    results = []
    for prob in probabilities:
        results.append([{ "label": labels[i], "score": float(prob[i]) } for i in range(len(labels))])

    return results
texts = ["This movie was great!", "I did not enjoy the film at all."]
results = classifier(texts)
print(results)
import shap

# Initialize the SHAP explainer
explainer = shap.Explainer(classifier, texts)

# Generate SHAP values for the first two texts
shap_values = explainer(texts[:2])

# Visualize SHAP explanations for the "POSITIVE" label (index 1)
shap.plots.text(shap_values[:, :, 1])
'''

In [ ]:
from transformers import pipeline, BertTokenizer, BertForSequenceClassification

model = BertForSequenceClassification.from_pretrained('/home/j597s263/bert_83')
tokenizer = BertTokenizer.from_pretrained('/home/j597s263/bert_83')

classifier = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer, return_all_scores=True)

texts = ["This movie was great!", "I did not enjoy the film at all."]
classifier(texts)

In [ ]:
import datasets
dataset = datasets.load_dataset("imdb", split="test")

In [ ]:
import shap

# define the explainer
explainer = shap.Explainer(classifier)

# explain the predictions of the pipeline on the first two samples
shap_values = explainer(texts)

shap.plots.text(shap_values[:, :, "LABEL_0"])

In [ ]:
# Define a single text to explain
single_text = ["This movie was great!"]

# Generate SHAP values for the single text
shap_values = explainer(single_text)


In [ ]:
shap.plots.text(shap_values[:, :, "LABEL_1"])

In [ ]:
# Initialize a dictionary to store aggregated SHAP values for each index
shap_dict = {i: 0.0 for i in range(512)}  

# Loop through the SHAP values and add them to the respective index in the dictionary
for i, shap_value in enumerate(shap_values.values[0]):
    if i < 512:  
        shap_dict[i] += float(shap_value[0])  

In [ ]:
 from datasets import load_dataset
import shap
from transformers import pipeline, BertTokenizer, BertForSequenceClassification

model = BertForSequenceClassification.from_pretrained('/home/j597s263/bert_83').to("cuda")
tokenizer = BertTokenizer.from_pretrained('/home/j597s263/bert_83')

classifier = pipeline(
    "sentiment-analysis",
    model=model,
    tokenizer=tokenizer,
    device=0,  
    truncation=True 
)

dataset = load_dataset("imdb", split="test")

explainer = shap.Explainer(classifier)

shap_dict = {i: 0.0 for i in range(512)}  # Dictionary with 512 indices initialized to 0.0

for i, example in enumerate(dataset['text']):
    shap_values = explainer([example])

    for idx, shap_value in enumerate(shap_values.values[0]):
        if idx < 512:  
            shap_dict[idx] += float(shap_value[0])  

    if (i + 1) % 100 == 0:
        print(f"Processed {i + 1} samples...")

print("Aggregated SHAP values:", shap_dict)
